In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib notebook
from  tqdm import tqdm_notebook
#import time

Getting MNIST using `torchvision.datasets`

In this case images are in [PIL](https://pillow.readthedocs.io/en/stable/) format, so we have to transform them to torch `tensor`

In [ ]:
import torchvision
mnist_train_data = torchvision.datasets.MNIST('dataset', train=True, download=True,
                                              transform=torchvision.transforms.ToTensor())
mnist_test_data = torchvision.datasets.MNIST('dataset', train=False, download=True,
                                             transform=torchvision.transforms.ToTensor())

image, label = mnist_train_data[0]
print(image.shape)  # Channels, width, height
print(type(image))

Alternative: Build dataset from numpy matrix with `TensorDataset`

In [ ]:
"""
from sklearn.datasets import fetch_openml
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)
mnist_train_data = TensorDataset(torch.from_numpy(X_train.astype('float32')),
                                 torch.from_numpy(y_train.astype('int64')))
"""

Inspecting the data

In [ ]:
fig, ax = plt.subplots(1, 10, figsize=(10, 2))
for k in range(10):
    image, label = mnist_train_data[k]
    #print(image.numpy().shape)
    ax[k].imshow(np.array(image.numpy()[0]), cmap=plt.cm.Greys_r)
    ax[k].axis('off')

Create Torch Dataloaders. This will push the data to the neural net during training

In [ ]:
from torch.utils.data import DataLoader
mnist_train_loader = DataLoader(mnist_train_data, batch_size=32, shuffle=True)

Create an artificial neural network model using PyTorch

In [ ]:
import torch
import torch.nn.functional as F

class Simple_convnet(torch.nn.Module):
    
    def __init__(self, n_hidden=10):
        super(Simple_convnet, self).__init__()
        #self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=n_conv, kernel_size=3, stride=1, bias=True)
        #self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        #self.conv2 = torch.nn.Conv2d(in_channels=n_conv, out_channels=n_conv, kernel_size=3, stride=1, bias=True)
        #self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = torch.nn.Linear(in_features=28*28,
                                   out_features=n_hidden, bias=True) #n_conv*5*5
        self.fc2 = torch.nn.Linear(in_features=n_hidden,
                                   out_features=10, bias=True)
        self.hidden_activation = torch.nn.ReLU()
        
    def forward(self, x):
        #z = F.relu(self.conv1(x.view(-1, 1, 28, 28)))
        #z = self.pool1(z)
        #z = F.relu(self.conv2(z))
        #z = self.pool2(z)
        #z = z.view(-1, net.fc1.in_features)
        z = x.view(-1, 28*28)
        z = self.hidden_activation(self.fc1(z))
        z = F.log_softmax(self.fc2(z), dim=1)
        return z

Perform training, we will use [hiddenlayer](https://github.com/waleedka/hiddenlayer) to track training variables

In [ ]:
import hiddenlayer as hl
my_conv_nnet = Simple_convnet()
# hl.build_graph(my_conv_nnet, torch.zeros([1, 3, 224, 224])) requires graphviz
# https://github.com/waleedka/hiddenlayer/blob/master/demos/pytorch_graph.ipynb

In [ ]:
my_conv_nnet = Simple_convnet()
nepochs = 5
optimizer = torch.optim.Adam(my_conv_nnet.parameters(), lr=1e-3)
criterion = torch.nn.NLLLoss()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = "cpu"
#net.to(device)

# Hiddenlayer objects to track metrics
history1 = hl.History()
canvas1 = hl.Canvas()

for epoch in range(nepochs): 
    epoch_loss, epoch_acc = 0.0, 0.0
    for data, target in mnist_train_loader:
        #data, target = data.to(device), target.to(device)
        prediction = my_conv_nnet(data)
        optimizer.zero_grad()        
        loss = criterion(prediction, target)  
        epoch_loss += loss.item()
        epoch_acc += (prediction.argmax(dim=1) == target).sum().item()
        if epoch > 0:
            loss.backward()
            optimizer.step()
    # Do this every X epochs:
    history1.log(epoch, loss=epoch_loss, accuracy=epoch_acc/mnist_train_data.__len__())
    with canvas1: # So that they render together
        canvas1.draw_plot([history1["loss"]])
        canvas1.draw_plot([history1["accuracy"]])
    #time.sleep(0.1)

### Tensorflow version

With TFrecord data format